# 12.1 FP树：用于编码数据集的有效方式

# 12.2 构建FP树

## 12.2.1 FP树的类定义

In [1]:
class tree_node:
    def __init__(self, name_value, num_occur, parent_node):
        self.name = name_value
        self.count = num_occur
        self.node_link = None
        self.parent = parent_node
        self.children = {}
    
    def inc(self, num_occur):
        self.count += num_occur
    
    def disp(self, ind=1):
        print('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [2]:
root_node = tree_node('pyramid', 9, None)

In [3]:
root_node.children['eye'] = tree_node('eye', 13, None)

In [4]:
root_node.disp()

   pyramid   9
     eye   13


In [5]:
root_node.children['phoenix'] = tree_node('phoenix', 3, None)

In [6]:
root_node.disp()

   pyramid   9
     eye   13
     phoenix   3


## 12.2.2 构建FP树

**程序清单12-2** FP树构建函数

In [7]:
def create_tree(data_set, min_sup=1):
    header_table = {}
    for trans in data_set:
        for item in trans:
            header_table[item] = header_table.get(item, 0) + data_set[trans]
    for k in list(header_table.keys()):
        if header_table[k] < min_sup:
            del(header_table[k])
    freq_item_set = set(header_table.keys())
    if len(freq_item_set) == 0:
        return None, None
    for k in header_table:
        header_table[k] = [header_table[k], None]
    ret_tree = tree_node('Null Set', 1, None)
    for tran_set, count in data_set.items():
        localD = {}
        for item in tran_set:
            if item in freq_item_set:
                localD[item] = header_table[item][0]
        if len(localD) > 0:
            ordered_items = [v[0] for v in sorted(localD.items(),
                                                 key=lambda p: p[1],
                                                 reverse=True)]
            update_tree(ordered_items, ret_tree, header_table, count)
    return ret_tree, header_table

In [8]:
def update_tree(items, in_tree, header_table, count):
    if items[0] in in_tree.children:
        in_tree.children[items[0]].inc(count)
    else:
        in_tree.children[items[0]] = tree_node(items[0], count, in_tree)
        if header_table[items[0]][1] == None:
            header_table[items[0]][1] = in_tree.children[items[0]]
        else:
            update_header(header_table[items[0]][1],
                         in_tree.children[items[0]])
    if len(items) > 1:
        update_tree(items[1::], in_tree.children[items[0]],
                   header_table, count)

In [9]:
def update_header(node_to_test, target_node):
    while (node_to_test.node_link != None):
        node_to_test = node_to_test.node_link
    node_to_test.node_link = target_node

**程序清单12-3** 简单数据集及数据包装器

In [10]:
def load_simp_dat():
    simp_dat = [['r','z','h','j','p'],
               ['z','y','x','w','v','u','t','s'],
               ['z'],
               ['r','x','n','o','s'],
               ['y','r','x','z','q','t','p'],
               ['y','z','x','e','q','s','t','m']]
    return simp_dat

In [11]:
def create_init_set(data_set):
    ret_dict = {}
    for trans in data_set:
        ret_dict[frozenset(trans)] = 1
    return ret_dict

In [12]:
simp_dat = load_simp_dat()
simp_dat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [13]:
init_set = create_init_set(simp_dat)
init_set

{frozenset({'z'}): 1,
 frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [14]:
my_FP_tree, my_header_tab = create_tree(init_set, 3)

In [15]:
my_FP_tree.disp()

   Null Set   1
     z   5
       r   1
       x   3
         y   3
           t   3
             s   2
             r   1
     x   1
       s   1
         r   1


In [16]:
my_header_tab

{'r': [3, <__main__.tree_node at 0x18f73c5aba8>],
 's': [3, <__main__.tree_node at 0x18f73c5a5f8>],
 't': [3, <__main__.tree_node at 0x18f73c5af60>],
 'x': [4, <__main__.tree_node at 0x18f73c5aa90>],
 'y': [3, <__main__.tree_node at 0x18f73c5aa58>],
 'z': [5, <__main__.tree_node at 0x18f73c5af98>]}

# 12.3 从一棵FP树中挖掘频繁项集

## 12.3.1 抽取条件模式基

**程序清单12-4** 发现以给定元素项结尾的所有路径的函数

In [17]:
def ascend_tree(leaf_node, prefix_path):
    if leaf_node.parent != None:
        prefix_path.append(leaf_node.name)
        ascend_tree(leaf_node.parent, prefix_path)

In [18]:
def find_prefix_path(base_pat, tree_node):
    cond_pats = {}
    while tree_node != None:
        prefix_path = []
        ascend_tree(tree_node, prefix_path)
        if len(prefix_path) > 1:
            cond_pats[frozenset(prefix_path[1:])] = tree_node.count
        tree_node = tree_node.node_link
    return cond_pats

In [19]:
find_prefix_path('x', my_header_tab['x'][1])

{frozenset({'z'}): 3}

In [20]:
find_prefix_path('z', my_header_tab['z'][1])

{}

In [21]:
find_prefix_path('r', my_header_tab['r'][1])

{frozenset({'z'}): 1,
 frozenset({'s', 'x'}): 1,
 frozenset({'t', 'x', 'y', 'z'}): 1}

## 12.3.2 创建条件FP树

**程序清单12-5** 递归查找频繁项集的`mine_tree`函数

In [40]:
def mine_tree(in_tree, header_table, min_sup, pre_fix, freq_item_list):
    bigL = [v[0] for v in sorted(header_table.items(), key=lambda p: str(p[1]))]
    for base_pat in bigL:
        new_freq_set = pre_fix.copy()
        new_freq_set.add(base_pat)
        freq_item_list.append(new_freq_set)
        cond_patt_bases = find_prefix_path(base_pat, header_table[base_pat][1])
        my_cond_tree, my_head = create_tree(cond_patt_bases, min_sup)
        if my_head != None:
            print('conditional tree for: ', new_freq_set)
            my_cond_tree.disp(1)
            mine_tree(my_cond_tree, my_head, min_sup, new_freq_set, freq_item_list)

In [41]:
freq_items = []

In [42]:
mine_tree(my_FP_tree, my_header_tab, 3, set([]), freq_items)

conditional tree for:  {'s'}
   Null Set   1
     x   3
conditional tree for:  {'y'}
   Null Set   1
     z   3
       x   3
conditional tree for:  {'y', 'x'}
   Null Set   1
     z   3
conditional tree for:  {'t'}
   Null Set   1
     y   3
       z   3
         x   3
conditional tree for:  {'t', 'z'}
   Null Set   1
     y   3
conditional tree for:  {'t', 'x'}
   Null Set   1
     y   3
       z   3
conditional tree for:  {'t', 'z', 'x'}
   Null Set   1
     y   3
conditional tree for:  {'x'}
   Null Set   1
     z   3


In [43]:
freq_items

[{'s'},
 {'s', 'x'},
 {'y'},
 {'y', 'z'},
 {'x', 'y'},
 {'x', 'y', 'z'},
 {'r'},
 {'t'},
 {'t', 'y'},
 {'t', 'z'},
 {'t', 'y', 'z'},
 {'t', 'x'},
 {'t', 'x', 'y'},
 {'t', 'x', 'z'},
 {'t', 'x', 'y', 'z'},
 {'x'},
 {'x', 'z'},
 {'z'}]

# 12.4 示例：在Twitter源中发现一些共现词

OK I will skip this part.

# 12.5 示例：从新闻网站点击流中挖掘

In [44]:
parsed_dat = [line.split() for line in open('kosarak.dat').readlines()]

In [45]:
init_set = create_init_set(parsed_dat)

In [46]:
my_FP_tree, my_header_tab = create_tree(init_set, 100000)

In [47]:
my_freq_list = []

In [48]:
mine_tree(my_FP_tree, my_header_tab, 100000, set([]), my_freq_list)

conditional tree for:  {'1'}
   Null Set   1
     6   107404
conditional tree for:  {'3'}
   Null Set   1
     6   186289
       11   117401
     11   9718
conditional tree for:  {'11', '3'}
   Null Set   1
     6   117401
conditional tree for:  {'11'}
   Null Set   1
     6   261773


In [49]:
len(my_freq_list)

9

In [50]:
my_freq_list

[{'1'},
 {'1', '6'},
 {'3'},
 {'11', '3'},
 {'11', '3', '6'},
 {'3', '6'},
 {'11'},
 {'11', '6'},
 {'6'}]

# 12.6 本章小结